In [1]:
import cv2
import numpy as np

# Load the image in grayscale
image = cv2.imread('floorPlan_0.png', cv2.IMREAD_GRAYSCALE)

threshold = 240  # Adjust this value as needed

# Create a 2D matrix of the same size as the image
height, width = image.shape
result_matrix = np.zeros((height, width), dtype=np.int32)

# Fill the cells based on the threshold
for i in range(height):
    for j in range(width):
        if image[i, j] < threshold:
            result_matrix[i, j] = 0
        else:
            result_matrix[i, j] = -1

#draw_matrix(result_matrix)
print("done")

done


In [2]:
import queue

def mark_pixel(x, y, value, pixel_queue):
    if x < 0 or y < 0 or x >= height or y >= width:
        return
    
    if result_matrix[x, y] != -1:
        return
    
    result_matrix[x, y] = value
    pixel_queue.put([x,y])

# Create a FIFO queue
pixel_queue = queue.Queue()

# enqueue all walls (v==0)

for i in range(height):
    for j in range(width):
        if result_matrix[i, j] == 0:
            pixel_queue.put([i, j])
            

# process each pixel
while not pixel_queue.empty():
    # print(pixel_queue.qsize(), " || ", i, "-", j)
    [i, j] = pixel_queue.get()
    current_value = result_matrix[i, j] + 1
    mark_pixel(i-1, j, current_value, pixel_queue)
    mark_pixel(i+1, j, current_value, pixel_queue)
    mark_pixel(i, j-1, current_value, pixel_queue)
    mark_pixel(i, j+1, current_value, pixel_queue)
    

print("done")

done


In [5]:
def print_matrix(matrix):
    for row in matrix:
        for element in row:
            print(element, end='  ')
        print()  # Move to the next line for the next row
    
# print_matrix(result_matrix)

In [11]:
def draw_matrix(image):
    new_img = np.array(image.copy())
    # Convert -1 values to 255 (white) for visualization
    new_img[new_img == -1] = 127
    
    new_img = new_img.astype(np.uint8)

    print(new_img.shape)
    # Combine the mask with the original image to draw the cells
    cv2.imshow("Grayscale Image", new_img)
    cv2.waitKey(0)  # This line allows you to close the image window by pressing any key
    cv2.destroyAllWindows()

In [6]:

cv2.imwrite("plan_0.png", result_matrix)

True

In [7]:
import random
import time


pivot_count = 0
def random_color():
    red = np.int32(random.randint(150, 255))
    green = np.int32(random.randint(150, 255))
    blue = np.int32(random.randint(150, 255))
    return (red, green, blue)

def is_vaild_index(index, verify_painted_matrix):
    [x, y] = index
    num_rows = len(verify_painted_matrix)
    num_cols = len(verify_painted_matrix[0])
    
    if not (0 <= x < num_rows and 0 <= y < num_cols):
        return False
    
    if verify_painted_matrix[x][y] == True:
        return False
    
    # in range, not painted
    return True
    
def is_not_all_painted(verify_painted_matrix):
    for row in verify_painted_matrix:
        for value in row:
            if value == False:
                return True
            
    return False

def find_max(distance_matrix, verify_painted_matrix):
    max_val = 0
    cor = [-1, -1]
    
    for i in range(len(distance_matrix)):
        for j in range(len(distance_matrix[0])):
            if verify_painted_matrix[i][j] == False:
                # not paint
                if distance_matrix[i][j] > max_val:
                    max_val = distance_matrix[i][j]
                    cor = [i, j]
    
    return cor

def paint_room(cordinate, distance_matrix, verify_painted_matrix, colored_image, count):
    print("paint room of: ",cordinate)
    [x, y] = cordinate
    
    color = (0, 0, 0)
    if distance_matrix[x][y] == 2:
        color = (0, 0, 0)
    else:
        color = random_color()
        
    
    room_pixel_queue = queue.Queue()
    room_pixel_queue.put(cordinate)
    
    r_val = 50 + count * 20
    if r_val > 255:
        r_val = 255
    colored_image[x][y] = (np.int32(0), np.int32(0), np.int32(r_val))
    verify_painted_matrix[x][y] = True
    count = 0
    while not room_pixel_queue.empty():
        cor = room_pixel_queue.get()
        # print(cor)
        x = cor[0]
        y = cor[1]
        curent_value = distance_matrix[x][y]
        count = count + 1
        
        x_n = x -1
        y_n = y
        if is_vaild_index([x_n, y_n], verify_painted_matrix):
            if distance_matrix[x_n][y_n] <= curent_value:
                # able to paint
                colored_image[x_n][y_n] = color
                verify_painted_matrix[x_n][y_n] = True
                room_pixel_queue.put([x_n, y_n])
                
        x_n = x + 1
        y_n = y
        if is_vaild_index([x_n, y_n], verify_painted_matrix):
            if distance_matrix[x_n][y_n] <= curent_value:
                # able to paint
                colored_image[x_n][y_n] = color
                verify_painted_matrix[x_n][y_n] = True
                room_pixel_queue.put([x_n, y_n])
                
        x_n = x
        y_n = y - 1
        if is_vaild_index([x_n, y_n], verify_painted_matrix):
            if distance_matrix[x_n][y_n] <= curent_value:
                # able to paint
                colored_image[x_n][y_n] = color
                verify_painted_matrix[x_n][y_n] = True
                room_pixel_queue.put([x_n, y_n])
                
        x_n = x
        y_n = y + 1
        if is_vaild_index([x_n, y_n], verify_painted_matrix):
            if distance_matrix[x_n][y_n] <= curent_value:
                # able to paint
                colored_image[x_n][y_n] = color
                verify_painted_matrix[x_n][y_n] = True
                room_pixel_queue.put([x_n, y_n])
    
    print(count)
        

def create_colored_room(distance_matrix, colored_image):
    height = len(distance_matrix)
    width = len(distance_matrix[0])
    
    verify_painted_matrix = [[False for _ in range(width)] for _ in range(height)]
    
    # fill wall with 2
    for i in range(height):
        for j in range(width):
            if result_matrix[i][j] == 0:
                result_matrix[i][j] = 2
                
    count = 0
    while is_not_all_painted(verify_painted_matrix):
        # find max
        cordinate = find_max(distance_matrix, verify_painted_matrix)
        paint_room(cordinate, distance_matrix, verify_painted_matrix, colored_image, count)
        count = count + 1
        print("one room done")
        
    
    print("all room done")
    return colored_image
    
    # locate the higest value

In [8]:
pivot_count = 0
colored_image = np.zeros((height, width, 3), dtype=np.uint8)
create_colored_room(result_matrix, colored_image)

print("das")
print(colored_image.shape)
print(colored_image[0][0])

paint room of:  [538, 234]
130370
one room done
paint room of:  [206, 217]
118330
one room done
paint room of:  [206, 637]
164624
one room done
paint room of:  [456, 571]
96810
one room done
paint room of:  [457, 570]
271
one room done
paint room of:  [458, 569]
269
one room done
paint room of:  [459, 568]
267
one room done
paint room of:  [460, 567]
265
one room done
paint room of:  [461, 566]
263
one room done
paint room of:  [462, 565]
261
one room done
paint room of:  [463, 564]
259
one room done
paint room of:  [464, 563]
257
one room done
paint room of:  [465, 562]
255
one room done
paint room of:  [466, 561]
253
one room done
paint room of:  [467, 560]
251
one room done
paint room of:  [468, 559]
5319
one room done
paint room of:  [449, 387]
6994
one room done
paint room of:  [456, 374]
51
one room done
paint room of:  [457, 373]
49
one room done
paint room of:  [458, 372]
47
one room done
paint room of:  [459, 371]
45
one room done
paint room of:  [460, 370]
43
one room done
pa

In [15]:

mark_path(result_matrix, colored_image)
cv2.imwrite("floor_01.png", colored_image)

True

In [14]:
def mark_path(result_matrix, colored_image):
    height = len(result_matrix)
    width = len(result_matrix[0])
    
    for i in range(height):
        for j in range(width):
            current_value = result_matrix[i, j]

            # Get the values of adjacent cells
            up_value = result_matrix[i - 1, j] if i > 0 else None
            down_value = result_matrix[i + 1, j] if i < height - 1 else None
            left_value = result_matrix[i, j - 1] if j > 0 else None
            right_value = result_matrix[i, j + 1] if j < width - 1 else None

            if (up_value is None or current_value >= up_value) and \
               (down_value is None or current_value >= down_value) and \
               (left_value is None or current_value >= left_value) and \
               (right_value is None or current_value >= right_value):
                colored_image[i][j] = (np.int32(0), np.int32(0), np.int32(0))